Preprocessing

In [1]:
import pandas as pd
import re

In [12]:
# Load dataset
file_path = 'Rawdata_1000.csv'
df = pd.read_csv(file_path)

# Data Cleaning
df = df[['Judul', 'Content']]
df.to_csv('step1_data_cleaning_result.csv', index=False)
print("hasil Step 1 Data Cleaning")
print(df.head())

hasil Step 1 Data Cleaning
                                               Judul  \
0  Wali Kota Malang Beri Dukungan dan Motivasi ke...   
1  Prabowo Beberkan Rencana Pengembangan Komodita...   
2  Curhat ke Rano Karno, Warga Condet: Selama Ini...   
3  Asas Praduga Tak Bersalah, Polri Belum Tetapka...   
4  Sidang Kasus Bentrok Amsterdam: 5 Orang Dihuku...   

                                             Content  
0  Baca berita dengan sedikit iklan,  klik di sin...  
1  Baca berita dengan sedikit iklan,  klik di sin...  
2   JAKARTA, KOMPAS.com - Warga Jalan Eretan 2, C...  
3   JAKARTA, KOMPAS.com - Bareskrim Polri belum m...  
4  Masih ingat bentrokan di Belanda pada November...  


In [13]:

def filtering_content(text):
    if not isinstance(text, str):
        return ""
    
    # 1. Standarisasi karakter (hapus non-breaking space)
    text = text.replace('\xa0', ' ')
    
    # 2. Hapus Iklan & Navigasi Spesifik
    junk_phrases = [
        "Baca berita dengan sedikit iklan,  klik di sini",
        "Baca berita dengan sedikit iklan, klik di sini",
        "Scroll ke bawah untuk melanjutkan membaca",
        "Ringkasan Berita",
        "INFO NASIONAL –",
        "INFO TEMPO –",
        "(*) "
    ]
    for phrase in junk_phrases:
        text = text.replace(phrase, " ")

    # 3. Hapus Header Media di awal teks (Lebih Fleksibel)
    # Menangani: "TEMPO.CO, Jakarta -", "Solo -", "JAKARTA, KOMPAS.com -"
    # Menggunakan \s* di depan untuk antisipasi spasi liar di awal
    text = re.sub(r'^\s*[A-Za-z0-9\.]+\s*,\s*[A-Za-z\s]+\s*[-–—]\s*', '', text) 
    text = re.sub(r'^\s*[A-Z\s]+,\s*[A-Za-z0-9\.]+\s*[-–—]\s*', '', text)
    text = re.sub(r'^\s*[A-Z][a-z]+\s*[-–—]\s*', '', text) # Contoh: Solo -
    text = re.sub(r'^\s*[A-Z\s]+\s*[-–—]\s*', '', text)    # Contoh: JAKARTA -

    # 4. Hapus Link Video & Pilihan Editor (beserta judul beritanya)
    # Menghapus frasa dan teks setelahnya sampai ketemu baris baru atau spasi ganda
    text = re.sub(r'(Simak|Lihat|Baca)\s+Juga\s+Video:?.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'Pilihan\s+Editor:?.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    
    # 5. Hapus Kredit Metadata (Editor, Penulis, Foto, dll)
    text = re.sub(r'(Video|Editor|Sumber|Foto|Videographer|Penulis|Reporter):\s*.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    
    # 6. Hapus Atribusi Foto/Kontributor
    text = re.sub(r'Dok\.\s+[A-Za-z\s]+', ' ', text)
    text = re.sub(r'[A-Z\s|]+\sberkontribusi dalam penulisan artikel ini\.', ' ', text)

    # 7. Bersihkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Terapkan pembersihan
df['Content'] = df['Content'].apply(filtering_content)

# Simpan hasil
df.to_csv('step2_filtering_result.csv', index=False)
print("hasil Step 2 Data Filtering")
print(df['Content'].head())

hasil Step 2 Data Filtering
0    Wali Kota Malang, Wahyu Hidayat, memberikan du...
1    Presiden Prabowo Subianto mengungkapkan rencan...
2    Warga Jalan Eretan 2, Condet, Jakarta Timur, m...
3    Bareskrim Polri belum menetapkan tersangka dal...
4    Masih ingat bentrokan di Belanda pada November...
Name: Content, dtype: object


In [15]:
df['Merged'] = df['Judul'] + " " + df['Content']

df['Merged'].to_csv('step3_Merged_result.csv', index=False)
print("hasil Step 3 Data Filtering")
print(df['Merged'].head())

hasil Step 3 Data Filtering
0    Wali Kota Malang Beri Dukungan dan Motivasi ke...
1    Prabowo Beberkan Rencana Pengembangan Komodita...
2    Curhat ke Rano Karno, Warga Condet: Selama Ini...
3    Asas Praduga Tak Bersalah, Polri Belum Tetapka...
4    Sidang Kasus Bentrok Amsterdam: 5 Orang Dihuku...
Name: Merged, dtype: object


In [11]:
# Case Folding 
df['Merged'] = df['Merged'].str.lower()

# Simpan hasil
df.to_csv('step4_Case_Folding_result.csv', index=False)
print("hasil Step 4 Case Folding")
print(df['Merged'].head())


hasil Step 4 Case Folding
0    wali kota malang beri dukungan dan motivasi ke...
1    prabowo beberkan rencana pengembangan komodita...
2    curhat ke rano karno  warga condet  selama ini...
3    asas praduga tak bersalah  polri belum tetapka...
4    sidang kasus bentrok amsterdam    orang dihuku...
Name: Merged, dtype: object


In [12]:
# Function removal
def remove_function(text):
    # hapus karakter selain huruf dan spasi
    text = re.sub(r'[^a-z\s]', ' ', text)
    return text

#Terapkan fungsi penghapusan karakter
df['Merged'] = df['Merged'].apply(remove_function)

#Simpan hasil
df.to_csv('step5_function_removal_result.csv', index=False)
print("hasil Step 5 Function Removal")
print(df['Merged'].head())

hasil Step 5 Function Removal
0    wali kota malang beri dukungan dan motivasi ke...
1    prabowo beberkan rencana pengembangan komodita...
2    curhat ke rano karno  warga condet  selama ini...
3    asas praduga tak bersalah  polri belum tetapka...
4    sidang kasus bentrok amsterdam    orang dihuku...
Name: Merged, dtype: object


In [14]:
# Normalisasi Teks 

# Definisikan kamus normalisasi
kamus_normalisasi = {
    # --- Institusi Pemerintahan, Hukum & Keamanan ---
    'kejagung': 'kejaksaan agung',
    'bareskrim': 'badan reserse kriminal',
    'pn': 'pengadilan negeri',
    'dpd': 'dewan perwakilan daerah',
    'dpr': 'dewan perwakilan rakyat',
    'dprd': 'dewan perwakilan rakyat daerah',
    'mpr': 'majelis permusyawaratan rakyat',
    'mk': 'mahkamah konstitusi',
    'ma': 'mahkamah agung',
    'kpk': 'komisi pemberantasan korupsi',
    'kpu': 'komisi pemilihan umum',
    'bawaslu': 'badan pengawas pemilihan umum',
    'bpbd': 'badan penanggulangan bencana daerah',
    'bpkp': 'badan pengawasan keuangan dan pembangunan',
    'bmkg': 'badan meteorologi klimatologi dan geofisika',
    'bin': 'badan intelijen negara',
    'mabes': 'markas besar',
    'pbb': 'perserikatan bangsa bangsa',
    'pemkot': 'pemerintah kota',
    'pemprov': 'pemerintah provinsi',
    'pemkab': 'pemerintah kabupaten',
    'pemda': 'pemerintah daerah',
    'kadinsos': 'kepala dinas sosial',
    'kadinkes': 'kepala dinas kesehatan',
    'kabag': 'kepala bagian',

    # --- Kepolisian & Militer ---
    'tni': 'tentara nasional indonesia',
    'polri': 'kepolisian negara republik indonesia',
    'polda': 'kepolisian daerah',
    'polres': 'kepolisian resor',
    'polsek': 'kepolisian sektor',
    'akbp': 'ajun komisaris besar polisi',
    'akp': 'ajun komisaris polisi',
    'tpnpb': 'tentara pembebasan nasional papua barat',

    # --- Kementerian & Pejabat ---
    'kemenag': 'kementerian agama',
    'kemendikbud': 'kementerian pendidikan dan kebudayaan',
    'kemenkes': 'kementerian kesehatan',
    'kemenkeu': 'kementerian keuangan',
    'kemenhub': 'kementerian perhubungan',
    'kemenlu': 'kementerian luar negeri',
    'kemhan': 'kementerian pertahanan',
    'menko': 'menteri koordinator',
    'mendagri': 'menteri dalam negeri',
    'pj': 'penjabat',
    'plt': 'pelaksana tugas',
    'pm': 'perdana menteri',
    'eks': 'mantan',
  

    # --- Partai Politik, Koalisi & Pemilu ---
    'pdip': 'partai demokrasi indonesia perjuangan',
    'pdi': 'partai demokrasi indonesia',
    'dpp': 'dewan pimpinan pusat',
    'pks': 'partai keadilan sejahtera',
    'pkb': 'partai kebangkitan bangsa',
    'ppp': 'partai persatuan pembangunan',
    'gol': 'golongan',
    'kim': 'koalisi indonesia maju',
    'munas': 'musyawarah nasional',
    'paslon': 'pasangan calon',
    'pilkada': 'pemilihan kepala daerah',
    'pemilu': 'pemilihan umum',
    'psu': 'pemungutan suara ulang',
    'cawagub': 'calon wakil gubernur',
    'cagub': 'calon gubernur',
    'cabup': 'calon bupati',
    'cawabup': 'calon wakil bupati',
    'tps': 'tempat pemungutan suara',

    # --- Istilah Birokrasi, Hukum & Tata Kota ---
    'kuhp': 'kitab undang undang hukum pidana',
    'ham': 'hak asasi manusia',
    'rt': 'rukun tetangga',
    'rw': 'rukun warga',
    'pp': 'peraturan pemerintah',
    'sk': 'surat keputusan',
    'uu': 'undang undang',
    'uud': 'undang undang dasar',
    'ruu': 'rancangan undang undang',
    'perpres': 'peraturan presiden',
    'asn': 'aparatur sipil negara',
    'pns': 'pegawai negeri sipil',
    'jl': 'jalan',

    # --- Istilah Umum, Ekonomi & Bisnis ---
    'umkm': 'usaha mikro kecil dan menengah',
    'bbm': 'bahan bakar minyak',
    'ppn': 'pajak pertambahan nilai',
    'psn': 'proyek strategis nasional',
    'tkp': 'tempat kejadian perkara',
    'thr': 'tunjangan hari raya',
    'hut': 'hari ulang tahun',
    'csr': 'corporate social responsibility',
    'tbk': 'terbuka',
    'pln': 'perusahaan listrik negara',
    'bumn': 'badan usaha milik negara',
    'bca': 'bank central asia',
    'pt': 'perseroan terbatas',
    'rp': 'rupiah',
    'sdm': 'sumber daya manusia',

    # --- Spesifik Konteks Berita, Tokoh & Ukuran ---
    'mbg': 'makan bergizi gratis',
    'gbk': 'gelora bung karno',
    'ikn': 'ibu kota nusantara',
    'dki': 'daerah khusus ibukota',
    'hp': 'handphone',
    'dr': 'dokter',
    'sby': 'susilo bambang yudhoyono',
    'ahy': 'agus harimurti yudhoyono',
    'rk': 'ridwan kamil',

    # --- Pendidikan, Kesehatan & Fasilitas ---
    'rsud': 'rumah sakit umum daerah',
    'rs': 'rumah sakit',
    'ponpes': 'pondok pesantren',
    'mts': 'madrasah tsanawiyah',
    'sd': 'sekolah dasar',
    'smp': 'sekolah menengah pertama',
    'sma': 'sekolah menengah atas',
    'smk': 'sekolah menengah kejuruan',

    # --- Kewarganegaraan & Kata Hubung/Chat ---
    'wn': 'warga negara',
    'wna': 'warga negara asing',
    'wni': 'warga negara indonesia',
    'ri': 'republik indonesia',
    'wib': 'waktu indonesia barat',
    'yg': 'yang',
    'dgn': 'dengan'
}

# Fungsi untuk melakukan normalisasi berdasarkan kamus
def normalize_text(text):
    words = str(text).split()
    
    # Mengubah kata jika terdapat di dalam kamus, jika tidak biarkan aslinya
    normalized_words = [kamus_normalisasi.get(word, word) for word in words]
    
    # Menggabungkan kembali list kata menjadi sebuah string teks kalimat
    return ' '.join(normalized_words)

#Terapkan fungsi normalisasi
df['Merged'] = df['Merged'].apply(normalize_text)

#Simpan hasil
df.to_csv('step6_normalization_result.csv', index=False)
print("hasil Step 6 Normalization")
print(df['Merged'].head())

hasil Step 6 Normalization
0    wali kota malang beri dukungan dan motivasi ke...
1    prabowo beberkan rencana pengembangan komodita...
2    curhat ke rano karno warga condet selama ini k...
3    asas praduga tidak bersalah kepolisian negara ...
4    sidang kasus bentrok amsterdam orang dihukum k...
Name: Merged, dtype: object


In [1]:
# Stopword Removal

In [ ]:
# Tokenisasi 

In [ ]:
# Stemming